In [ ]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt


import matplotlib.pyplot as plt
%matplotlib inline


from glob import glob

In [ ]:
all_q = {}
x_dirs = glob('yz/*/')

x_dirs[0].split('/')


'1qtable'.split('1')

In [ ]:
for x_dir in x_dirs:
    chain_length = x_dir.split('/')[1]
    qtables = glob(f'{x_dir}{chain_length}*')
    print(qtables)
    all_q[chain_length] = {}
    for qtable in qtables:
        spacing = qtable.split(f'{x_dir}{chain_length}')[1].split('qtable')[0]
        
        
        with open(qtable) as fp:
            #The first 14 lines of the qTable do not contain spectrum data
            print(qtable)
            for blank in range(0,14):
                fp.readline()
            wave = []
            Q_ext = []
            Q_abs = []
            Q_sca = []
            for k in range(350,801):
                line = fp.readline()
                ary = line.split(" ")
                ary = [a for a in ary if a]
#                 print(ary[1:5])
                ary = np.array(ary[1:5]).astype(np.float)
                wave.append(float(ary[0]))
                Q_ext.append(float(ary[1]))
                Q_abs.append(float(ary[2]))
                Q_sca.append(float(ary[3]))

                
            df = pd.DataFrame({'wave': wave, 'Q_ext': Q_ext, 'Q_abs': Q_abs, 'Q_sca': Q_sca})
            all_q[chain_length][spacing] = df

In [ ]:
from scipy.interpolate import UnivariateSpline
unreg = all_q['24']['1'].dropna()
spl = UnivariateSpline(unreg['wave'], unreg['Q_ext'])

wl = np.arange(0.350, 0.800, 0.001)
# inp = ((wl - w_mean)/w_std).reshape(-1, 1)

spl.set_smoothing_factor(0.00001)
preds = spl(wl) 

In [ ]:
plt.plot(all_q['24']['1']['wave'], all_q['24']['1']['Q_ext'], 'g')
plt.plot(wl, preds, 'b')

In [ ]:
all_q['24']['1'].loc[all_q['24']['1']['Q_ext'].isnull(), 'Q_ext'] 

In [ ]:
preds[all_q['24']['1']['Q_ext'].isnull()]

In [ ]:
for n in all_q:
    for spacing in all_q[n]:
        
        df = all_q[n][spacing]
        df_copy = df.dropna()
        spl = UnivariateSpline(np.array(df_copy['wave']), np.array(df_copy['Q_abs']))
        wl = np.arange(0.350, 0.800, 0.001)
        spl.set_smoothing_factor(0.000001)
        preds = spl(wl) 

        df.loc[df['Q_ext'].isnull(), 'Q_ext'] = preds[df['Q_ext'].isnull()]
        
        all_q[n][spacing] = df

In [ ]:
all_q['5']['1'][350:370]

In [ ]:
df_list = {}
for n in all_q:
    n_list = []
    for spacing in all_q[n]:
        
        cp = all_q[n][spacing].copy()
        cp['spacing'] = float(spacing)
        n_list.append(cp)
    
    df = pd.concat(n_list, axis=0)
    df_list[n] = df
    
df_list['3'].head()

In [ ]:
formatted_df = {}

for n in df_list:
    df = df_list[n]
    new_df = pd.DataFrame()
    for space in [1.0, 2.0, 3.0, 4.0]:
        ser = df.loc[df['spacing'] == space, 'Q_ext']
        if not ser.empty:
            new_df[str(space)] = ser
        
    formatted_df[n] = new_df

In [ ]:
df = df_list['5']
new_df = pd.DataFrame()
for space in [1.0, 2.0, 3.0, 4.0]:
    ser = df.loc[df['spacing'] == space, 'Q_ext']
    if not ser.empty:
        new_df[str(space)] = ser

In [ ]:
df = formatted_df['5']
df[350:370]

In [ ]:
for i in range(0, 451):
    print(i)
    print(df.loc[i])

In [ ]:
from scipy import interpolate
x = {}

for n in range(2,36):
    df = formatted_df[str(n)]
    
    y = []
    print(n)
    for i in range(0, 451):
        columns = np.array(df.columns).astype(np.float)
        vals = np.array(df.loc[i])
        f = interpolate.interp1d(columns, vals, kind='quadratic', fill_value='extrapolate')
        df_out = f(np.arange(0.8, 4.05, 0.05))
        y.append(df_out)
        
    y = np.array(y)
    x[n] = y

In [ ]:
def mapper(inp):
    return '%.2f' % (0.8 + 0.05 * float(inp))

final = {}

for n in x:
    d = pd.DataFrame(x[n])
    d = d.rename(columns=mapper)
    wl_df = pd.DataFrame({'wl':np.arange(.350, .800, .001)})
    out = wl_df.join(d)
    print(out)
    out.to_csv(f'yz_{n}_new_interp.csv')

In [ ]:
from scipy.interpolate import BivariateSpline
from scipy import interpolate


ones = df_list[0][df_list[0]['spacing'] == 1.0].dropna()
twos = df_list[0][df_list[0]['spacing'] == 2.0]
threes = df_list[0][df_list[0]['spacing'] == 3.0]
fours = df_list[0][df_list[0]['spacing'] == 4.0]


# spl = BivariateSpline(ones['wave'], ones['spacing'], ones['Q_abs'], s=0.000001)


# tck = interpolate.bisplrep(ones['wave'], ones['spacing'], ones['Q_abs'], s=0.1)
# znew = interpolate.bisplev(ones['wave'], ones['spacing'], tck)

# wl = np.arange(0.350, 0.800, 0.001)

# preds = spl(ones['wave'], ones['spacing']) 
plt.plot(ones['wave'], ones['Q_abs'])
plt.plot(twos['wave'], twos['Q_abs'])
plt.plot(threes['wave'], threes['Q_abs'])
plt.plot(fours['wave'], fours['Q_abs'])


# plt.plot(ones['wave'], znew)

In [ ]:
spl = UnivariateSpline([1.0, 2.0, 3.0, 4.0], [ones['Q_abs'][180], twos['Q_abs'][180], threes['Q_abs'][180], fours['Q_abs'][180]])

In [ ]:
spl.set_smoothing_factor(0.01)
plt.plot([1.0, 2.0, 3.0, 4.0], [ones['Q_abs'][180], twos['Q_abs'][180], threes['Q_abs'][180], fours['Q_abs'][180]])
plt.plot([1.0, 2.0, 3.0, 4.0], [spl(1.0), spl(2.0), spl(3.0), spl(4.0)])

In [ ]:
df_list[0]